# Sample IUCN range maps

Samples a set of species from locally stored IUCN range maps and then samples random points to generate a set of presence and absence points.

## Methodology

Species range data is held locally (could adapt to use IUCN Red List API).


## Setup

### Library import


In [11]:
import geopandas as gpd
from shapely.geometry import Point, box, Polygon
import random
import numpy as np
import pandas as pd
import os
from EcoNameTranslator import EcoNameTranslator
from EcoNameTranslator import ReverseTranslator

### Utils


In [3]:

def read_species_subset(shapefile_path,feature_class = None, n_species_sample = 20):
    # Load the shapefile using geopandas
    if feature_class:
        species_ranges = gpd.read_file(shapefile_path,layer = feature_class)
    else:
        species_ranges = gpd.read_file(shapefile_path)

    print("Read species ranges")
    #select extant or possibly extant species where the 'PRESENCE' attribute is either 1 or 3
    filtered_species_ranges = species_ranges[species_ranges['presence'].isin([1, 2, 3])]
    print("Filtered for extant and possibly extant")
    filtered_species_ranges = filtered_species_ranges[filtered_species_ranges['seasonal'].isin([1, 2, 3])]
    print("Filtered for seasonality - removing passage")

    # Randomly sample a subset of species ranges
    if n_species_sample < len(species_ranges):
        sampled_species_ranges = species_ranges.sample(n=n_species_sample, random_state=42)  # Set random_state for reproducibility
    else:
        sampled_species_ranges = species_ranges  # If n_species_sample exceeds the number of available ranges, take all ranges
    
    print("Sampled points")

    # Join polygons with the same 'BINOMIAL' (species name) together
    if 'binomial' in sampled_species_ranges.columns:
        sampled_species_ranges = sampled_species_ranges.dissolve(by='binomial')
    elif 'sci_name' in sampled_species_ranges.columns:
        sampled_species_ranges = sampled_species_ranges.dissolve(by='sci_name')
        sampled_species_ranges.rename(columns={'sci_name':'binomial'})
    print("Dissolved ranges")
    return sampled_species_ranges


# Create a function to randomly generate points within the bounding box
def generate_random_point_within_bbox(bbox):
    minx, miny, maxx, maxy = bbox
    x = random.uniform(minx, maxx)
    y = random.uniform(miny, maxy)
    return Point(x, y)

def sample_points_presence_absence(species_range,bounds, n_presence = 10,n_absence = 10):
    # Sample points and classify them as presence/absence
    #n_points = 1000  # Number of points to sample

    points = []
    presence_absence = []

    presences = species_range.sample_points(size = n_presence)

    # Create the inverse polygon (area outside the species range within the bounding box)
    bbox_polygon = box(*bounds)
    inverse_polygon = bbox_polygon.difference(species_range.geometry)
    #print(type(inverse_polygon))
    #absence_range_gdf = gpd.GeoSeries(inverse_polygon, crs=species_range.crs)

    absences = inverse_polygon.sample_points(size = n_absence)
    
    pdf = presences.get_coordinates()
    pdf['presence'] = np.ones(pdf.shape[0])


    adf = absences.get_coordinates()
    adf['presence'] = np.zeros(adf.shape[0])



    return pd.concat([adf,pdf],axis = 0)

<a id='Read subset of species'></a>
## Read species 

Read species shapefile/gdb and extract a subset of the species 


In [4]:
#Mammals
# define the mammals shapefile
mammal_shapefile_path = "/mnt/c/Users/mikeha/Work/Spatial data/Red List/2022/MAMMALS/MAMMALS.shp"
mammal_subset = read_species_subset(mammal_shapefile_path)


Read species ranges
Filtered for extant and possibly extant
Filtered for seasonality - removing passage
Sampled points
Dissolved ranges


In [5]:
reptile_shapefile_path = "/mnt/c/Users/mikeha/Work/Spatial data/Red List/2022/REPTILES/REPTILES.shp"
reptile_subset = read_species_subset(reptile_shapefile_path)

Read species ranges
Filtered for extant and possibly extant
Filtered for seasonality - removing passage
Sampled points
Dissolved ranges


In [118]:
amphibian_shapefile_path = "/mnt/c/Users/mikeha/Work/Spatial data/Red List/2022/AMPHIBIANS/AMPHIBIANS.shp"
amphibian_subset = read_species_subset(amphibian_shapefile_path)

Read species ranges
Filtered for extant and possibly extant
Filtered for seasonality - removing passage
Sampled points
Dissolved ranges


In [7]:
bird_shapefile_path = "/mnt/c/Users/mikeha/Work/Spatial data/Red List/2022/Birds/batch_1.shp"
bird_subset = read_species_subset(bird_shapefile_path)
bird_subset.index.names = ['binomial']

Read species ranges
Filtered for extant and possibly extant
Filtered for seasonality - removing passage
Sampled points
Dissolved ranges


<a id='Sample points'></a>
## Sample points for presences/absences




In [85]:
def construct_p_a_points(subset):
    bounds = subset.total_bounds  # [minx, miny, maxx, maxy]

    all_df_pa = None
    for i in range(subset.shape[0]):
        s_df_pa = sample_points_presence_absence(subset.iloc[[i]],bounds)
        if all_df_pa is None:
            all_df_pa = s_df_pa
        else:
            all_df_pa = pd.concat([all_df_pa,s_df_pa])
    
    return all_df_pa


def get_common_names(scientific_names):
    translator = ReverseTranslator()
    common_names = translator.translate(scientific_names)
    list_common_names = []
    for binomial in scientific_names:
        if len(common_names[binomial][1]) > 0:
            list_common_names.append(common_names[binomial][1][0])
        else:
            list_common_names.append("")

    return list_common_names

In [30]:
mammal_points = construct_p_a_points(mammal_subset)
mammal_points['common'] = get_common_names(scientific_names=mammal_points.index.values)

Beginnging name reformatting
Name reformatting complete
Validating scientific names
Expanding higher level taxonomic names
Trying common name translation...(this may take a while)


In [86]:
reptile_points = construct_p_a_points(reptile_subset)
reptile_points['common'] = get_common_names(scientific_names=reptile_points.index.values)

Beginnging name reformatting
Name reformatting complete
Validating scientific names
Expanding higher level taxonomic names
Trying common name translation...(this may take a while)


In [119]:
amphibian_points = construct_p_a_points(amphibian_subset)
amphibian_points['common'] = get_common_names(amphibian_points.index.values)

Beginnging name reformatting
Name reformatting complete
Validating scientific names
Expanding higher level taxonomic names
Trying common name translation...(this may take a while)


In [95]:
bird_points = construct_p_a_points(bird_subset)
bird_points['common'] = get_common_names(bird_points.index.values)

Beginnging name reformatting
Name reformatting complete
Validating scientific names
Expanding higher level taxonomic names
Trying common name translation...(this may take a while)


In [101]:
#Deal with any missing common names
def find_empty_common_names(points):
    return np.unique(points.iloc[np.where(points['common'] == "")[0]].index.values)



In [98]:
find_empty_common_names(mammal_points)

array([], dtype=object)

In [102]:
find_empty_common_names(reptile_points)

array(['Aipysurus mosaicus', 'Aristelliger praesignis',
       'Calotes bhutanensis', 'Cyrtodactylus srilekhae',
       'Cyrtodactylus wallacei', 'Hebius sarawacensis',
       'Hemidactylus mindiae', 'Homopholis fasciata',
       'Lamprophis erlangeri', 'Liolaemus ceii', 'Liolaemus isabelae',
       'Phalotris mertensi', 'Plestiodon japonicus'], dtype=object)

In [112]:
replacements = {
    'Aipysurus mosaicus':'mosaic sea snake',
    'Aristelliger praesignis':'croaking lizard',
       'Calotes bhutanensis':'Bhutan Beaty Lizard',
       'Cyrtodactylus srilekhae': 'Bangalore Geckoella',
       'Cyrtodactylus wallacei':'South Sulawesi Bent-toed Gecko',
       'Hebius sarawacensis':'Sarawak Keelback',
       'Hemidactylus mindiae':'Mount Sinai Gecko',
       'Homopholis fasciata':'Banded Velvet Gecko',
       'Lamprophis erlangeri':'Ethiopian House Snake',
       'Liolaemus ceii':'Ceis Tree Iguana',
       'Liolaemus isabelae':'Isabels Tree Iguana',
       'Phalotris mertensi':'Coral-Falsa',
       'Plestiodon japonicus':'Japanese skink'
}

for binomial in replacements:
    reptile_points.loc[binomial,'common'] = replacements[binomial]

#Check no empty common names
find_empty_common_names(reptile_points)

In [120]:
find_empty_common_names(amphibian_points)

array(['Ameerega yoshina', 'Ansonia minuta', 'Austrochaperina polysticta',
       'Bolitoglossa vallecula', 'Laliostoma labrosum',
       'Leptobrachella picta', 'Limnonectes gyldenstolpei',
       'Limnonectes ingeri', 'Litoria multicolor', 'Litoria singadanae',
       'Megophrys palpebralespinosa', 'Proceratophrys bigibbosa',
       'Rana japonica', 'Scutiger boulengeri', 'Speleomantes supramontis',
       'Xenorhina macrops'], dtype=object)

In [122]:
replacements = {
    'Ameerega yoshina':'poison dart frog',
    'Ansonia minuta':'Tiny Stream Toad',
    'Austrochaperina polysticta':'Morobe Land Frog',
    'Bolitoglossa vallecula':'Yarmal Mushroom-tongue Salamander',
    'Laliostoma labrosum':'Madagascar Bullfrog',
    'Leptobrachella picta':'Painted Slender Litter Frog',
    'Limnonectes gyldenstolpei':'Capped Frog',
    'Limnonectes ingeri':'Ingers Wart Frog',
    'Litoria multicolor':'Multi-coloured Tree Frog',
    'Litoria singadanae':'Green tree frog',
    'Megophrys palpebralespinosa':'Rough-skinned Horned Toad',
    'Proceratophrys bigibbosa':'Peters Smooth Horned Frog',
    'Rana japonica':'Japanese brown frog',
    'Scutiger boulengeri':'Xizang Alpine Toad',
    'Speleomantes supramontis':'Supramonte Cave Salamander',
    'Xenorhina macrops':'Hellwig Fanged Frog'
}

for binomial in replacements:
    amphibian_points.loc[binomial,'common'] = replacements[binomial]

#Check no empty common names
find_empty_common_names(amphibian_points)

array([], dtype=object)

In [123]:
find_empty_common_names(bird_points)

array(['Calliope obscura', 'Hydrobates hornbyi'], dtype=object)

In [124]:
replacements = {
    'Calliope obscura':'black-throated robin',
    'Hydrobates hornbyi':'Ringed storm petrel'
}

for binomial in replacements:
    bird_points.loc[binomial,'common'] = replacements[binomial]

#Check no empty common names
find_empty_common_names(bird_points)

array([], dtype=object)

In [125]:
all_df_pa = pd.concat([mammal_points, reptile_points, amphibian_points, bird_points])

In [126]:
all_df_pa['x'] = np.round(all_df_pa['x'],decimals = 2)
all_df_pa['y'] = np.round(all_df_pa['y'],decimals = 2)

In [127]:
#write the dataframe to file
all_df_pa.to_csv('../eval/species_point_presence_absence.csv')

<a id='conclusion'></a>
## Conclusion

{ Summarize findings and next steps. }
